Ensembling with simple weighted average and linear regression. The first level models are lgb, xgb and neural network.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, LinearRegression
import gc

### Read second level features (first level model predictions) and targets from disk

In [2]:
data = pd.read_pickle('../output/data.pkl')
Y_train_level2 = data[data.date_block_num == 33]['item_cnt_month']
del data
gc.collect()

7

In [3]:
X_train_level2 = pd.DataFrame()
df = pd.read_csv('../output/lgb_valid.csv')
X_train_level2['lgb'] = df['item_cnt_month']
df = pd.read_csv('../output/xgb_valid.csv')
X_train_level2['xgb'] = df['item_cnt_month'] 
df = pd.read_csv('../output/nn_valid.csv')
X_train_level2['nn'] = df['item_cnt_month'] 

X_test_level2 = pd.DataFrame()
df = pd.read_csv('../output/lgb_submission.csv')
X_test_level2['lgb'] = df['item_cnt_month']
df = pd.read_csv('../output/xgb_submission.csv')
X_test_level2['xgb'] = df['item_cnt_month'] 
df = pd.read_csv('../output/nn_submission.csv')
X_test_level2['nn'] = df['item_cnt_month'] 

### fit second level model

In [4]:
# simple weighted average
# find best linear combination coefficient to weight 
best_alpha = 1;
best_rmse = 100;
for alpha in np.arange(0,1,0.02):
    Y_pred_level2 = alpha*X_train_level2['lgb'] + (1-alpha)*X_train_level2['xgb']
    rmse = np.sqrt(mean_squared_error(Y_train_level2, Y_pred_level2))
    if (rmse<best_rmse):
        best_rmse = rmse
        best_alpha = alpha

Y_test_level2 = best_alpha*X_test_level2['lgb'] + (1-best_alpha)*X_test_level2['xgb']
print('best alpha:', best_alpha)
print('weighted average of lgb and xgb validation rmse: ',best_rmse)
submission = pd.DataFrame({
    "ID": np.arange(Y_test_level2.shape[0]), 
    "item_cnt_month": Y_test_level2
})
submission.to_csv('../output/blended_submission1.csv', index=False)

# Linear regression
model = LinearRegression()
model.fit(X_train_level2, Y_train_level2)
Y_pred_level2 = model.predict(X_train_level2)
Y_test_level2 = model.predict(X_test_level2)
rmse = np.sqrt(mean_squared_error(Y_train_level2, Y_pred_level2))
print('Linear regression validation rmse: ',rmse)
submission = pd.DataFrame({
    "ID": np.arange(Y_test_level2.shape[0]), 
    "item_cnt_month": Y_test_level2
})
submission.to_csv('../output/blended_submission2.csv', index=False)

best alpha: 0.84
weighted average of lgb and xgb validation rmse:  0.888324003454071


/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


Linear regression validation rmse:  0.8863685906116213


Public score:

weighted average of lgb and xgb: 0.90338

linear regression: 0.90255